<a href="https://colab.research.google.com/github/joaoSouza2121/Embargos-Declaracao/blob/main/Embargos_de_Declaracao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Inteligência Artificial para Previsão de Sentenças em Embargos de Declaração

In [1]:
# Imports
import torch
import torch.nn as nn
import numpy as np

##Preparando os Dados
O texto abaixo é um exemplo de embargo de declaração. Embora o texto represente um embargo, dados críticos foram substituídos por informações genéricas, o que não compromete o objetivo do estudo de caso.

In [2]:
# Texto de embargo de declaração
embargo = """O embargante sofreu o ajuizamento de ação de danos morais e materiais, cujo objeto é o reaver os
valores pagos pelo sinal dado em um contrato de compra e venda de imóvel no qual não foi dado continuidade. Em 24
de fevereiro de 2012, o Magistrado proferiu decisão de fls. 277 a 280, que condenou todas as demandadas
solidariamente no seguinte teor: Diante de todo o exposto, com fundamento no art. 1234, I, do CPC/2015,
julgo procedentes em parte os pedidos constantes na inicial, condenando solidariamente as demandadas, XPTO LTDA,
BOB CAMARGO DE MORAES, a Pagarema título de indenização por danos morais, consoante fundamentação acima discorrida,
o montante de R$ 1.500,00 (um mil e quinhentos reais), corrigidos monetariamente pelo INPC desde a data
desta decisão, acrescidos de juros de 1% ao mês, a partir da citação; condeno ainda, à restituição do valor
pago pelo demandante como sinal da entrada do imóvel, descontando apenas 20% (vinte por cento), referente às
despesas, devendo incidir juros de 1% (um por cento) ao mês contados da citação e correção monetária pelo INPC a
partir da sentença. Contudo, data venia, houve omissão e obscuridade na referida decisão, haja vista que a omissão
se deu pela ausência dos julgamentos das preliminares (Necessidade de Perícia Técnica e a incompetência de
Juizado Especial) proposta posteriormente em aditamento de contestação (Fls 251 a 254) para impugnar áudios
juntados pelo embargado, autorizado a ser realizada pela Douta Magistrada em audiência de Conciliação,
instrução e julgamento de fls 235 e 236, por ausência de intimação anterior para realizar a já tratada
impugnação aos áudios anexados."""


In [3]:
# Limpeza do texto substituindo vírgulas e pontos por espaços e colocando as palavras em minúsculo
embargo = embargo.replace(',','').replace('.','').lower().split()


In [4]:
# Criação do corpus com o texto acima
corpus = set(embargo)

In [5]:
# Visualiza o corpus
corpus

{'(fls',
 '(necessidade',
 '(um',
 '(vinte',
 '1%',
 '1234',
 '150000',
 '20%',
 '2012',
 '235',
 '236',
 '24',
 '251',
 '254)',
 '277',
 '280',
 'a',
 'acima',
 'acrescidos',
 'aditamento',
 'ainda',
 'ajuizamento',
 'anexados',
 'anterior',
 'ao',
 'aos',
 'apenas',
 'art',
 'as',
 'audiência',
 'ausência',
 'autorizado',
 'ação',
 'bob',
 'camargo',
 'cento)',
 'citação',
 'citação;',
 'com',
 'como',
 'compra',
 'conciliação',
 'condenando',
 'condeno',
 'condenou',
 'consoante',
 'constantes',
 'contados',
 'contestação',
 'continuidade',
 'contrato',
 'contudo',
 'correção',
 'corrigidos',
 'cpc/2015',
 'cujo',
 'da',
 'dado',
 'danos',
 'das',
 'data',
 'de',
 'decisão',
 'demandadas',
 'demandante',
 'descontando',
 'desde',
 'despesas',
 'desta',
 'deu',
 'devendo',
 'diante',
 'discorrida',
 'do',
 'dos',
 'douta',
 'e',
 'em',
 'embargado',
 'embargante',
 'entrada',
 'especial)',
 'exposto',
 'fevereiro',
 'fls',
 'foi',
 'fundamentação',
 'fundamento',
 'haja',
 'houve',
 

In [6]:
# Comprimento do corpus
corpus_length = len(corpus)

In [7]:
# Dicionários para TF-IDF
dic_palavra = {}
dic_inverso_palavra = {}

In [8]:
# Loop pelo corpus para criar os dicionários
for i, palavra in enumerate(corpus):
    dic_palavra[palavra] = i
    dic_inverso_palavra[i] = palavra

In [9]:
# Lista para receber os dados
dados = []

In [10]:
# Loop pelo texto par extrair sentenças e palavras
for i in range(2, len(embargo) - 2):
    sentence = [embargo[i-2], embargo[i-1], embargo[i+1], embargo[i+2]]
    target = embargo[i]
    dados.append((sentence, target))


sentence = [embargo[i-2], embargo[i-1], embargo[i+1], embargo[i+2]]

Para uma palavra no índice i, obtemos duas palavras antes e duas palavras depois. A palavra no índice i será o nosso target e a sentença será composta das duas palavras e duas palavras depois da palavra target.

Após treinar o modelo, seremos capazes de prever cada palavra com base nas palavras a sua volta.

In [11]:
# Visualizado os dados
print(dados[3])

(['o', 'ajuizamento', 'ação', 'de'], 'de')


As quatro palavras na lista serão os dados de entrada e a palavra fora da lista, sera a variavel de saida.('de' nesse caso)

# Construção do Modelo CBoW

In [12]:
# definindo o comprimento de cada embedding
embedding_length = 20

In [13]:
# Classe para o modelo
class CBoW(torch.nn.Module):

    # Método construtor
    def __init__(self, corpus_length, embedding_dim):
        super(CBoW, self).__init__()
       
        # Camada de entrada do modelo para criação da embedding
        self.embeddings = nn.Embedding(corpus_length, embedding_dim)

        # Camadas lineares
        self.linear1 = nn.Linear(embedding_dim, 64)
        self.linear2 = nn.Linear(64, corpus_length)
       
        # Camadas de ativação
        self.activation_function1 = nn.ReLU()
        self.activation_function2 = nn.LogSoftmax(dim = -1)

    # Passo (forward)
    def forward(self, inputs):
       
        # Aqui definimos a ordem ds camadas da rede neural
        embeds = sum(self.embeddings(inputs)).view(1,-1)
        out = self.linear1(embeds)
        out = self.activation_function1(out)
        out = self.linear2(out)
        out = self.activation_function2(out)
        return out

    # Obtém a word_emdedding
    def get_word_emdedding(self, word):
        word = torch.LongTensor([dic_palavra[word]])
        return self.embeddings(word).view(1,-1)


In [14]:
# Cria o modelo CBoW
modelo = CBoW(corpus_length, embedding_length)


In [15]:
# Função de custo
loss_function = nn.NLLLoss()


In [16]:
# Otimizador do modelo (backpropagation)
optimizer = torch.optim.SGD(modelo.parameters(), lr = 0.01)


In [17]:
# Função para criar o vetor de sentenças, necessário para treinar o modelo
def make_sentence_vector(sentence, word_dict):
    idxs = [word_dict[w] for w in sentence]
    return torch.tensor(idxs, dtype = torch.long)

In [18]:
# Dicionario de palavras
dic_palavra

{'(fls': 176,
 '(necessidade': 152,
 '(um': 147,
 '(vinte': 102,
 '1%': 12,
 '1234': 53,
 '150000': 93,
 '20%': 155,
 '2012': 42,
 '235': 37,
 '236': 134,
 '24': 92,
 '251': 57,
 '254)': 61,
 '277': 26,
 '280': 31,
 'a': 27,
 'acima': 25,
 'acrescidos': 100,
 'aditamento': 98,
 'ainda': 124,
 'ajuizamento': 141,
 'anexados': 80,
 'anterior': 97,
 'ao': 87,
 'aos': 71,
 'apenas': 16,
 'art': 6,
 'as': 172,
 'audiência': 175,
 'ausência': 107,
 'autorizado': 66,
 'ação': 10,
 'bob': 11,
 'camargo': 43,
 'cento)': 131,
 'citação': 30,
 'citação;': 59,
 'com': 112,
 'como': 106,
 'compra': 32,
 'conciliação': 151,
 'condenando': 127,
 'condeno': 39,
 'condenou': 85,
 'consoante': 99,
 'constantes': 49,
 'contados': 126,
 'contestação': 156,
 'continuidade': 19,
 'contrato': 69,
 'contudo': 165,
 'correção': 84,
 'corrigidos': 89,
 'cpc/2015': 9,
 'cujo': 36,
 'da': 164,
 'dado': 74,
 'danos': 20,
 'das': 153,
 'data': 154,
 'de': 46,
 'decisão': 75,
 'demandadas': 104,
 'demandante': 48,
 

In [19]:
# O dicionário de palavras será convertido em um vetor de sentenças. Aqui um exemplo:
print(make_sentence_vector(['pela','ausência','dos','julgamentos'], dic_palavra))


tensor([101, 107, 120, 171])


##Treinamento do Modelo

In [20]:
# Loop por 150 passadas (epochs) de treinamento
for epoch in range(150):
   
    # Inicia o erro da época com 0
    epoch_loss = 0
   
    # Loop pelos dados de entrada (sentence) e saída (target)
    for sentence, target in dados:
       
        # Inicializa os gradientes com zero
        modelo.zero_grad()
       
        # Cria o vetor de sentença com os dados de entrada (que devem estar no dicionário de palavras)
        sentence_vector = make_sentence_vector(sentence, dic_palavra)  
       
        # Usa o vetor para fazer previsões com o modelo e retorna as probabilidades
        log_probs = modelo(sentence_vector)
       
        # Calcula o erro do modelo
        loss = loss_function(log_probs, torch.tensor([dic_palavra[target]], dtype = torch.long))
       
        # Chama o método de backpropagation para calcular o gradiente da derivada
        loss.backward()
       
        # Otimiza os pesos do modelo e segue para a próxima passada
        # É aqui que o aprendizado acontece
        optimizer.step()
       
        # Atualiza o erro da época
        epoch_loss += loss.data
       
    # Imprime epoch e erro da epoch    
    print('Epoch: ' + str(epoch) + ', Erro do Modelo: ' + str(epoch_loss.item()))


Epoch: 0, Erro do Modelo: 1355.1697998046875
Epoch: 1, Erro do Modelo: 1234.663818359375
Epoch: 2, Erro do Modelo: 1141.6375732421875
Epoch: 3, Erro do Modelo: 1049.6793212890625
Epoch: 4, Erro do Modelo: 952.3583374023438
Epoch: 5, Erro do Modelo: 847.2706909179688
Epoch: 6, Erro do Modelo: 736.2577514648438
Epoch: 7, Erro do Modelo: 622.0828247070312
Epoch: 8, Erro do Modelo: 510.283447265625
Epoch: 9, Erro do Modelo: 406.8824768066406
Epoch: 10, Erro do Modelo: 316.377197265625
Epoch: 11, Erro do Modelo: 241.63351440429688
Epoch: 12, Erro do Modelo: 183.13949584960938
Epoch: 13, Erro do Modelo: 139.2725372314453
Epoch: 14, Erro do Modelo: 106.71981811523438
Epoch: 15, Erro do Modelo: 83.20346069335938
Epoch: 16, Erro do Modelo: 65.71538543701172
Epoch: 17, Erro do Modelo: 52.82475662231445
Epoch: 18, Erro do Modelo: 43.15243148803711
Epoch: 19, Erro do Modelo: 35.98662185668945
Epoch: 20, Erro do Modelo: 30.576839447021484
Epoch: 21, Erro do Modelo: 26.408767700195312
Epoch: 22, Err

O Erro foi reduzido a cada passada do aprendizado

# Previsão

In [21]:
# Função para obter uma previsão
def get_resultado_previsto(input, dic_inverso_palavra):
    index = np.argmax(input)
    return dic_inverso_palavra[index]


In [22]:
# Função para prever sentenças (aplicamos aos novos dados o mesmo tratamento usado nos dados de treino)
def preve_sentenca(sentence):
   
    # Dividimos a sentença com split
    sentence_split = sentence.replace('.','').lower().split()
   
    # Criamos o vetor de sentença
    sentence_vector = make_sentence_vector(sentence_split, dic_palavra)
   
    # Faz a previsão com o modelo
    prediction_array = modelo(sentence_vector).data.numpy()
   
    # Print dos resultados
    print('Palavras Anteriores: {}\n'.format(sentence_split[:2]))
    print('Palavra Prevista: {}\n'.format(get_resultado_previsto(prediction_array[0], dic_inverso_palavra)))
    print('Palavras Seguintes: {}\n'.format(sentence_split[2:]))


##Previsões com o Modelo
Dentro da frase: "ausência de intimação anterior para realizar", vejamos se o modelo consegue prever a palavra.

Vou omitir a palavra intimação e essa deve ser a palavra prevista pelo modelo. Vamos passar como dados de entrada as duas palavras anteriores e as duas palavras posteriores.

In [23]:
preve_sentenca('ausência de anterior para')

Palavras Anteriores: ['ausência', 'de']

Palavra Prevista: intimação

Palavras Seguintes: ['anterior', 'para']



In [25]:
# Embedding da palavra
print(modelo.get_word_emdedding('intimação'))

tensor([[-0.5104, -0.0242, -2.1160,  0.2460,  0.1927, -0.3201, -0.2984, -0.2386,
         -1.0195, -1.2949, -1.5453,  0.0362, -0.2132, -1.3972,  0.3941, -0.1989,
          0.3803, -0.4413, -0.2238, -0.6779]], grad_fn=<ViewBackward>)


In [26]:
# Previsão com o modelo
preve_sentenca('devendo incidir de 1%')

Palavras Anteriores: ['devendo', 'incidir']

Palavra Prevista: juros

Palavras Seguintes: ['de', '1%']

